# 文本预处理



In [1]:
import collections
import re
from d2l import torch as d2l

将数据集读取到由多条文本行组成的列表中

In [2]:
d2l.DATA_HUB['time_machine'] = (d2l.DATA_URL + 'timemachine.txt',
                                '090b5e7e70c295757f55df93cb0a180b9691891a')

def read_time_machine():  
    """将时间机器数据集加载到文本行的列表中"""
    with open(d2l.download('time_machine'), 'r') as f:
        lines = f.readlines()
        # 非英文字符全变为空格
    return [re.sub('[^A-Za-z]+', ' ', line).strip().lower() for line in lines]

lines = read_time_machine()
print(f"{lines[0]}")
print(lines[0])
print(lines[10])

the time machine by h g wells
the time machine by h g wells
twinkled and his usually pale face was flushed and animated the


每个文本序列又被拆分成一个词元列表

In [3]:
def tokenize(lines, token='word'):  
    # token的作用就是划分每个词
    # 中文的就是划分每个字
    """将文本行拆分为单词或字符词元"""
    if token == 'word':
        return [line.split() for line in lines]
    elif token == 'char':
        return [list(line) for line in lines]
    else:
        print('错误：未知词元类型：' + token)

tokens = tokenize(lines,'word')
for i in range(11):
    print(tokens[i])

['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
[]
[]
[]
[]
['i']
[]
[]
['the', 'time', 'traveller', 'for', 'so', 'it', 'will', 'be', 'convenient', 'to', 'speak', 'of', 'him']
['was', 'expounding', 'a', 'recondite', 'matter', 'to', 'us', 'his', 'grey', 'eyes', 'shone', 'and']
['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']


构建一个字典，通常也叫做*词表*（vocabulary），
用来将字符串类型的词元映射到从$0$开始的数字索引中

In [4]:
class Vocab:  
    """文本词表"""
    # 将每个词映射数字索引
    # reserved_tokens句子开始/结束的token
    def __init__(self, tokens=None, min_freq=0, reserved_tokens=None):
        if tokens is None:
            tokens = []
        if reserved_tokens is None:
            reserved_tokens = []
            # 每个次出现的次数
        counter = count_corpus(tokens)
        self._token_freqs = sorted(counter.items(), key=lambda x: x[1],
                                   reverse=True)
        self.idx_to_token = ['<unk>'] + reserved_tokens
        self.token_to_idx = {token: idx
                             for idx, token in enumerate(self.idx_to_token)}
        # 根据频数，确定每个词的索引
        for token, freq in self._token_freqs:
            if freq < min_freq:
                break
            if token not in self.token_to_idx:
                # 词
                self.idx_to_token.append(token)
                # 词-索引
                self.token_to_idx[token] = len(self.idx_to_token) - 1

    def __len__(self):
        return len(self.idx_to_token)

    def __getitem__(self, tokens):
        if not isinstance(tokens, (list, tuple)):
            # 尝试从字典self.token_to_idx中获取键为tokens的值。
            # 如果tokens在字典中，则返回对应的值；否则，返回字典中的默认值self.unk。
            return self.token_to_idx.get(tokens, self.unk)
        return [self.__getitem__(token) for token in tokens]

    def to_tokens(self, indices):
        if not isinstance(indices, (list, tuple)):
            return self.idx_to_token[indices]
        return [self.idx_to_token[index] for index in indices]
# @property: 这是一个内置的Python装饰器，它被用来改变一个方法的行为，使其成为一个特殊的"属性"。
# 使属性转换为**只读**属性，让属性可以通过方法的方式访问，可以像访问属性一样来访问方法,无需再加括号调用,更简洁。
# 当你调用一个对象的方法时，你会得到一个结果。然而，使用@property装饰器，你可以使一个方法看起来像一个属性，当你访问这个"属性"时，你实际上是在调用这个方法。
    @property  #只读属性
    def unk(self):
        return 0
    
# 你实际上是在获取`_token_freqs`的值。
    @property
    def token_freqs(self):
        return self._token_freqs

def count_corpus(tokens):  
    """统计词元的频率"""
    # 它接受两个参数:

# object:要判断类型的对象
# classinfo:可以是类型对象,也可以是类型对象组成的元组
    if len(tokens) == 0 or isinstance(tokens[0], list):
        tokens = [token for line in tokens for token in line]
    return collections.Counter(tokens)

构建词表

In [5]:
vocab = Vocab(tokens)
print(len(vocab))
print(list(vocab.token_to_idx.items())[:10])

4580
[('<unk>', 0), ('the', 1), ('i', 2), ('and', 3), ('of', 4), ('a', 5), ('to', 6), ('was', 7), ('in', 8), ('that', 9)]


将每一条文本行转换成一个数字索引列表

In [6]:
for i in [0, 10]:
    print(i)
    print('文本:', tokens[i])
    print('索引:', vocab[tokens[i]])

0
文本: ['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
索引: [1, 19, 50, 40, 2183, 2184, 400]
10
文本: ['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']
索引: [2186, 3, 25, 1044, 362, 113, 7, 1421, 3, 1045, 1]


将所有功能打包到`load_corpus_time_machine`函数中

In [8]:
def load_corpus_time_machine(max_tokens=-1):  
    """返回时光机器数据集的词元索引列表和词表"""
    lines = read_time_machine()
    # 词的类别索引
    tokens = tokenize(lines, 'char')
    vocab = Vocab(tokens)
    # 一长串词语的整数，对应索引
    corpus = [vocab[token] for line in tokens for token in line]
    if max_tokens > 0:
        corpus = corpus[:max_tokens]
    return corpus, vocab

corpus, vocab = load_corpus_time_machine()
len(corpus), len(vocab)

(170580, 28)